In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchain_groq langchain-openai langchain-experimental neo4j wikipedia tiktoken yfiles_jupyter_graphs

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.7/301.7 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━

In [3]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [16]:
groq_api_key = "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"
NEO4J_URI="neo4j+s://0ea7e7ee.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="hPDx7s6tmiz-DUyTjFD6te1Sik0ZfL37QXi9tmbgZRw"

In [5]:
from typing import Tuple, List, Optional
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import ConfigurableField
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase

In [6]:
from langchain_community.vectorstores import Neo4jVector

In [7]:
import os
os.environ["GROQ_API_KEY"] = groq_api_key
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD



In [8]:
from langchain_community.graphs import Neo4jGraph

In [9]:
graph = Neo4jGraph()

<ipython-input-9-6e7385464d00>:1: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph()


In [58]:
from langchain.document_loaders import WikipediaLoader
raw_documents = WikipediaLoader(query="Typhoid disease").load()

In [59]:
from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(raw_documents[:3])

In [60]:
print(len(documents))

6


In [61]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0,
    max_tokens=2048,
    timeout=None,
    max_retries=2,
    groq_api_key = groq_api_key
)
print("model retrieved")

model retrieved


In [62]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm)

In [63]:
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [64]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [65]:
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

In [66]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase

In [67]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

In [68]:
def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    display(widget)
    return widget


In [69]:

showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

GraphWidget(layout=Layout(height='800px', width='100%'))

In [89]:
!pip install langchain_google_genai

In [98]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from google.colab import userdata
google_api_key = userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_API_KEY"] = google_api_key
vector_index = Neo4jVector.from_existing_graph(
    GoogleGenerativeAIEmbeddings(model="models/embedding-001"),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [99]:
graph.query("CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

[]

In [100]:
from langchain_core.pydantic_v1 import BaseModel, Field
# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )

In [101]:

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [102]:

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)


In [103]:
entity_chain = prompt | llm.with_structured_output(Entities)


entity_chain.invoke({"question": "Explain malaria to me as if i am 8 yo"}).names

['organization', 'person']

In [104]:
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

In [105]:
def generate_full_text_query(input: str) -> str:
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()



# Fulltext index query
def structured_retriever(question: str) -> str:
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [106]:
print(structured_retriever("How to treat malaria?"))

Malaria - CAUSED_BY -> Mosquito
Malaria - TREATS -> Quinine
Malaria - TREATS -> Doxycycline
Malaria - TREATS -> Artemisinin
Malaria - TREATS -> Chloroquine
Malaria - ORIGIN -> Africa
Malaria - RECORDED_HISTORY -> Greece
Malaria - RECORDED_HISTORY -> China
Malaria - TREATMENT -> Quinine
Malaria - TREATMENT -> Chloroquine
Malaria - TREATMENT -> Cinchona Tree
Malaria - TREATMENT -> Artemisinins
Malaria - TRANSMISSION -> Mosquito
Malaria - CAUSATION -> Plasmodium
Malaria - CONTROL_MEASURE -> Ddt
Malaria - IMPACT -> U.S.
Malaria - IMPACT -> South Pacific
Malaria - ENDEMIC -> Central America
Malaria - ENDEMIC -> South America
Malaria - ENDEMIC -> Hispaniola
Malaria - ENDEMIC -> Middle East
Malaria - ENDEMIC -> Indian Subcontinent
Malaria - ENDEMIC -> Southeast Asia
Malaria - ENDEMIC -> Oceania
Malaria - TRANSMITTED_BY -> Mosquito
Rosenblum - TREATED_WITH -> Malaria
Julius Wagner-Jauregg - DEVELOPED_THERAPY -> Malaria
P. Falciparum - CAUSED_BY -> Malaria
P. Vivax - CAUSED_BY -> Malaria
P. Kno

In [107]:
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    return final_data

In [108]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""


CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)


def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer


_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatGroq(temperature=0)
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x : x["question"]),
)


template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""


prompt = ChatPromptTemplate.from_template(template)


chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [109]:
import warnings
warnings.filterwarnings("ignore")

In [110]:

chain.invoke({"question": "Explain malaria to me as if i am 8 yo"})

Search query: Explain malaria to me as if i am 8 yo


"Hey there! So, you know how sometimes we get sick and feel yucky? Malaria is a kind of sickness that some people get. It's caused by tiny bugs called mosquitoes that bite us and make us sick.\n\nWhen a mosquito bites us, it gives us tiny parasites that make us feel sick. We might get a fever, feel tired, and even get headaches. If we don't get treated, the sickness can get worse and even be very serious.\n\nBut don't worry! There are ways to prevent getting malaria, like using special nets and sprays to keep the mosquitoes away. And if we do get sick, there are medicines that can help us feel better.\n\nMalaria is a big problem in some parts of the world, but we can all help by being careful and taking steps to prevent it."